<a href="https://colab.research.google.com/github/HELPMUMFRONTIER/nig_translation/blob/main/train_model_to_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install datasets transformers[sentencepiece]
!apt install git-lfs
!pip install numpy
!pip install transformers
!pip install sacrebleu
!pip install accelerate
!pip install tqdm
!pip install huggingface_hub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.3 MB/s eta 0:00:00


In [ ]:
from transformers import M2M100Tokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric, Dataset
import huggingface_hub #import notebook_login
import numpy as np
import torch
import pandas as pd

In [ ]:
huggingface_hub.login()

In [ ]:
from datasets import load_dataset


datasets = load_dataset("HelpMum-Personal/nigeria_translation")#, lang1="en", lang2="fr",trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1380000 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9131 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'language'],
        num_rows: 1380000
    })
    val: Dataset({
        features: ['translation', 'language'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['translation', 'language'],
        num_rows: 9131
    })
})

In [ ]:
metric = load_metric("sacrebleu",trust_remote_code=True)

model_checkpoint = "facebook/m2m100_418M"

<ipython-input-7-c96286475f25>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu",trust_remote_code=True)


In [ ]:
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint)

max_input_length = 128
max_target_length = 128

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#tokenizer.src_lang = "en"
tokenizer.tgt_lang = "en"

def preprocess_function(examples):
    #tokenizer.src_lang = "yo"
    src_langs=[ex for ex in examples["language"]]
    tokenizer.src_lang = src_langs[0]
    print(tokenizer.src_lang)
    #print(src_langs)
    inputs = [ex[src_langs[i]] for i, ex in enumerate(examples["translation"])]
    #inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    #print(inputs)

    #tmps_index = 0
    #for i in inputs:
    #  tmps_index += 1

    #  if tmps_index >= 1000:
    #    inputs.remove(i)

    #tmps_index = 0
    #for i in targets:
    #  tmps_index += 1

    #  if tmps_index >= 1000:
    #   targets.remove(i)

    model_inputs = tokenizer(inputs, text_target=targets,max_length=max_input_length, truncation=True)
    #print(model_inputs[0])
    # Set up the tokenizer for targets
    #with tokenizer.as_target_tokenizer():
    #    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    #model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
    batch_size=20000,
)

Map:   0%|          | 0/1380000 [00:00<?, ? examples/s]

yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ig
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

yo
yo
ig
ig
ha
ha


Map:   0%|          | 0/9131 [00:00<?, ? examples/s]

yo


In [ ]:
tokenized_datasets=tokenized_datasets.shuffle(seed=42)

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    #labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
del datasets
#del datasets

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
#batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
huggingface_hub.login()

In [ ]:
YOUR_HF_USERNAME="HelpMum-Personal"
args = Seq2SeqTrainingArguments(
    output_dir = f"{YOUR_HF_USERNAME}/m2m100_418M-nig-en",
    #output_dir = f"{YOUR_HF_USERNAME}/m2m100_418M-yo-en",
    evaluation_strategy="no",
    save_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    #save_steps = 200,
    logging_steps = 500,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_accumulation_steps=3,
    predict_with_generate=True,
    push_to_hub=True,
    do_train=True,
    do_eval=True,
    fp16=True,
    report_to="tensorboard",
    save_steps = 10000,
    eval_steps = 10000,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Step,Training Loss
500,1.939600
1000,1.944900
1500,1.914600
2000,1.891300
2500,1.889000
3000,1.863900
3500,1.862700
4000,1.856500
4500,1.824200
5000,1.820300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


Step,Training Loss
500,1.939600
1000,1.944900
1500,1.914600
2000,1.891300
2500,1.889000
3000,1.863900
3500,1.862700
4000,1.856500
4500,1.824200
5000,1.820300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


TrainOutput(global_step=21563, training_loss=1.8480164226660596, metrics={'train_runtime': 6439.7248, 'train_samples_per_second': 214.295, 'train_steps_per_second': 3.348, 'total_flos': 3.5779980533170176e+17, 'train_loss': 1.8480164226660596, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


events.out.tfevents.1725714963.e42eb3dd7ee5.3168.1:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

events.out.tfevents.1725716814.e42eb3dd7ee5.3168.2:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

events.out.tfevents.1725716857.e42eb3dd7ee5.3168.3:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

events.out.tfevents.1725714887.e42eb3dd7ee5.3168.0:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HelpMum-Personal/m2m100_418M-nig-en/commit/92b7a2e55f5db52b9c9d9feaa9bc07d992ddfa53', commit_message='Training complete', commit_description='', oid='92b7a2e55f5db52b9c9d9feaa9bc07d992ddfa53', pr_url=None, pr_revision=None, pr_num=None)